<a href="https://colab.research.google.com/github/sssuinlee/BDP_project/blob/main/rnn_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import random
import numpy as np
import pandas as pd

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import OneHotEncoder

In [4]:
#csv 읽어오기
train = pd.read_csv("/content/drive/MyDrive/OTTO/train_prep.csv")
train = train.drop(train.columns[[0]],axis = 1)
display(train)

,customer_id,product_code,time_stamp,event_type
0,0,1517085,2022-07-31 22:00:00.025,clicks
1,0,1563459,2022-07-31 22:01:44.511,clicks
2,0,1309446,2022-08-01 15:23:59.426,clicks
3,0,16246,2022-08-01 15:28:39.997,clicks
4,0,1781822,2022-08-01 15:31:11.344,clicks
...,...,...,...,...
10285556,199999,928064,2022-08-01 06:46:48.967,clicks
10285557,199999,849970,2022-08-01 06:47:29.078,clicks
10285558,199999,1052480,2022-08-01 06:49:07.035,clicks
10285559,199999,487255,2022-08-01 06:49:21.116,clicks


In [5]:
#원핫인코더 객체 생성
ohe = OneHotEncoder(handle_unknown="ignore")

In [6]:
#customers = session0~session8 을 이어붙인 dataframe
n_customers = 8
customers=train[train.customer_id<=n_customers]
customers

,customer_id,product_code,time_stamp,event_type
0,0,1517085,2022-07-31 22:00:00.025,clicks
1,0,1563459,2022-07-31 22:01:44.511,clicks
2,0,1309446,2022-08-01 15:23:59.426,clicks
3,0,16246,2022-08-01 15:28:39.997,clicks
4,0,1781822,2022-08-01 15:31:11.344,clicks
...,...,...,...,...
827,7,1064510,2022-08-15 04:41:58.319,clicks
828,8,1814223,2022-07-31 22:00:00.136,clicks
829,8,324620,2022-07-31 22:00:15.389,clicks
830,8,1320098,2022-07-31 22:00:27.550,clicks


In [7]:
#pid_etype = session8 까지의 pid와 etype
pid_etype = pd.DataFrame(data={"product_code":customers.product_code, "event_type":customers.event_type})
#session8 까지의 데이터로 원핫인코딩 dictionary 생성
ohe.fit(pid_etype)

OneHotEncoder(handle_unknown='ignore')

In [8]:
#sessions = session8까지, [각 session의 pid와 etype]. 
#그러니까 sessions 라는 리스트 안에, session 이라는 리스트가 9개
sessions =[]
for i in range (n_customers):
    customer = customers[customers.customer_id==i]
    pid_etype = pd.DataFrame(data={"product_code":customer.product_code, "event_type":customer.event_type})
    sessions.append(pid_etype)

In [9]:
#5개씩 자르는 batchmaker
#각 session 별로 5줄을 한 input batch(덩어리)로 묶어 input_batch 에
#그 다음 줄은 target에 넣고 ohe 해서 target_batch 에
#input_batch, target_batch 는 ohe 한걸 가지고 있음

sequence_length=5     #5줄씩

def batchmaker(sessions):
    input_batch = []
    target_batch = []
    target = pd.DataFrame(data={"product_code":[], "event_type":[]})
    for session in sessions:
        for i in range(len(session)-sequence_length):
            input_batch.append(ohe.transform(session[i:i+5]).toarray())
            target = target.append(session[i+5:i+6])
            target_batch.append(ohe.transform(session[i+5:i+6]).toarray())
    return input_batch, target_batch, target

In [10]:
input_batch, target_batch, target = batchmaker(sessions)

In [11]:
input_batch = torch.tensor(input_batch, dtype=torch.float32, requires_grad = True)
target_batch = torch.tensor(target_batch, dtype=torch.float32).squeeze()
target.product_code = target.product_code.astype(int)

<ipython-input-11-a60194afe4bf>:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  input_batch = torch.tensor(input_batch, dtype=torch.float32, requires_grad = True)


In [12]:
##hyper parameters

hidden_size = 2    #(pid,etype)
#batch_size = len(input_batch)
learning_rate = 0.01
input_size=len(ohe.categories_[0]) + len(ohe.categories_[1])
#sequence_length = 5 (위에서 정의했음)
num_layers = 1

In [13]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)   #input(batch_size, seq_len, input_size)
        self.weights = nn.Linear(hidden_size, input_size, bias=False)
        self.bias = nn.Parameter(torch.zeros(input_size))
        self.Softmax = nn.Softmax(dim=1)
        
        
    def forward(self, hidden, X):           
        X = X.transpose(0, 1)              #X: [n_step, batch_size, n_feature]
        outputs, hidden = self.rnn(X, hidden)
        outputs = outputs[-1]                 # 최종 예측 Hidden Layer [batch_size, num_directions*hidden_size]
        model = self.weights(outputs) + self.bias  # 최종 예측 최종 출력 층
        return model
    
model = RNN(input_size, hidden_size, num_layers)

#Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
#feature 500개일때 5000epoch: 9초(3.14>2.6>1.6>1.43>1.41)
for epoch in range(5000):
    #print('epoch: ', epoch+1)
    #hidden 을 0으로 초기화. 학습으로 변경가능
    hidden = torch.zeros(1, sequence_length, hidden_size, requires_grad=True)   
    #input_batch 에 가중치 hidden을 써서 나온 output
    output = model(hidden, input_batch)    
    #criterion 
    #print(output.shape, target_batch.shape)
    loss = criterion(output, target_batch)    #criterion(예측치, 답), loss 를 계산. 
    if (epoch + 1) % 1000 == 0:      #99, 199, 299, 399, 499 번째 학습이면 loss 를 출력하도록
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss)) 
    #backward and optimize
    optimizer.zero_grad()   #gradient(미분값)가 누적되지 않도록 0으로 초기화
    loss.backward()         #cost function 미분해서 gradient(현지점 기울기) 계산
    optimizer.step()        #weight, bias 업데이트

In [ ]:
hidden = torch.zeros(1, sequence_length, hidden_size, requires_grad=True)   

In [ ]:
predict =  model(hidden, input_batch).data.topk(k=2)[1]
print('predict', predict)

In [ ]:
target_transform = ohe.transform(target)

In [ ]:
#target_list : 정답
import numpy
target_list = []
for i in range(target_transform.shape[0]):
    target_list.append(list(numpy.where(target_transform.toarray()[i]==1))[0].tolist())
target_list

In [ ]:
#predict_list : 예측
predict_list = []
for i in range(len(predict)):
    predict_list.append(predict[i].sort().values.tolist())
predict_list

In [ ]:
##정답이랑 예측 비교해서 채점하는 recall 함수 만들어야
target_list[0]